In [9]:
import yfinance as yf
import pandas as pd




[*********************100%***********************]  1 of 1 completed


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,005930.KS,005930.KS,005930.KS,005930.KS,005930.KS,005930.KS
Date,,,,,,
2021-01-04,77225.750000,83000.0,84400.0,80200.0,81000.0,38655276
2021-01-05,78063.148438,83900.0,83900.0,81600.0,81600.0,35335669
2021-01-06,76481.398438,82200.0,84500.0,82100.0,83300.0,42089013
2021-01-07,77132.695312,82900.0,84200.0,82700.0,82800.0,32644642
2021-01-08,82622.250000,88800.0,90000.0,83000.0,83300.0,59013307


In [15]:
stock_dict={"Gold":'132030.KS', "KTB":"148070.KS", "200TR":"278530.KS", "USTB": "304660.KS", "Sp500":"360750.KS"}

In [100]:
for key, value in stock_dict.items():
    # 삼성전자 주식 데이터 다운로드
    df = yf.download(value, '2003-01-01', '2024-10-30')

    # datetime 열의 시간대 정보 제거
    df.index = df.index.tz_localize(None)

    # 수정된 데이터프레임을 Excel과 CSV 파일로 저장
    df.to_csv(f"ISA/{key}.csv")



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [176]:
import pandas as pd

def load_data(file_path):
    """CSV 파일에서 필요한 열을 읽어옵니다."""
    df = pd.read_csv(file_path, index_col='Price', parse_dates=True)
    return df[['High', 'Low']]

def generate_investment_dates(start_date, end_date):
    """매월 15일을 기준으로 한 투자 날짜 목록을 생성합니다."""
    investment_dates = pd.date_range(start=start_date, end=end_date, freq='MS') + pd.DateOffset(days=14)
    return pd.to_datetime(investment_dates.strftime('%Y-%m-%d'))

def get_investment_data(df, investment_dates):
    """매월 15일 이후 가장 가까운 영업일의 데이터를 가져옵니다."""
    investment_data = pd.DataFrame()
    for date in investment_dates:
        while date.strftime('%Y-%m-%d') not in df.index:
            date += pd.DateOffset(days=1)  # 다음 날로 이동
        closest_date_data = df.loc[[date.strftime('%Y-%m-%d')]]
        investment_data = pd.concat([investment_data, closest_date_data])
    return investment_data

def calculate_investment(investment_data, monthly_investment=1000000):
    """매월 평균 가격으로 주식 수를 계산하고 총 투자 금액과 총 주식 수를 반환합니다."""
    investment_data['Average Price'] = investment_data[['High', 'Low']].apply(pd.to_numeric, errors='coerce').mean(axis=1)
    investment_data['Shares Bought'] = monthly_investment / investment_data['Average Price']
    total_investment = monthly_investment * len(investment_data)
    total_shares = investment_data['Shares Bought'].sum()
    return total_investment, total_shares

def calculate_final_value(df, total_shares, final_date):
    """최종 평가 금액을 계산합니다. final_date 이후 가장 가까운 영업일을 사용합니다."""
    # 날짜가 없으면 그 이후 가장 가까운 영업일로 이동
    # 날짜가 없으면 그 이후 가장 가까운 영업일로 이동
    while final_date not in df.index:
        final_date = (pd.to_datetime(final_date) + pd.DateOffset(days=1)).strftime('%Y-%m-%d')  # 다음 날로 이동
    
    # 최종 가격을 사용하여 평가 금액 계산
    final_price = pd.to_numeric(df.loc[final_date, 'Low'], errors='coerce')
    final_value = total_shares * final_price
    return final_value


def calculate_profit_rate(final_value, total_investment):
    """수익률을 계산합니다."""
    profit_rate = (final_value - total_investment) / total_investment * 100
    return profit_rate

def main(file_path, final_date, money, start_date, end_date):
    # 단계별 함수 실행
    df = load_data(file_path)
    investment_dates = generate_investment_dates(start_date, end_date)
    investment_data = get_investment_data(df, investment_dates)
    total_investment, total_shares = calculate_investment(investment_data, monthly_investment=money)
    final_value = round(calculate_final_value(df, total_shares, final_date), 2)
    profit_rate = round(calculate_profit_rate(final_value, total_investment), 2)
    
    # 결과 출력
    print(f"총 투자금액: {total_investment:.2f}원")
    print(f"{final_date} 기준 총 평가 금액: {final_value:.2f}원")
    print(f"{final_date} 기준 수익률: {profit_rate:.2f}%")
    
    return total_investment,final_value


In [200]:
# 사용 예시

def 회계(num, half):
    if half=="5_10":
        final_date = f'20{num}-10-20'
        start_date = f'20{num}-05-01'
        end_date = f'20{num}-10-30'
        stock_rate_dict={"Gold":15, "KTB":32.5, "200TR":10, "USTB": 32.5, "Sp500":10}
    if half=="11_4":
        final_date = f'20{num+1}-4-20'
        start_date = f'20{num}-11-01'
        end_date = f'20{num+1}-4-30'
        stock_rate_dict={"Gold":15, "KTB":17.5, "200TR":25, "USTB": 17.5, "Sp500":25}
        
    investment_dict = {}
    value_dict = {}

    for key, value in stock_rate_dict.items():
        print(f"-------------{key}------------")
        
        investment_dict[key],value_dict[key] = main(f"ISA/{key}.csv", final_date, value, start_date, end_date)

    # value_dict의 모든 값을 더하여 sum_total에 저장
    sum_total_investment = sum(investment_dict.values())
    sum_total_value = sum(value_dict.values())
    print(f"모든 종목의 투자 금액 합계: {sum_total_investment:.2f}원")
    print(f"모든 종목의 최종 평가 금액 합계: {sum_total_value:.2f}원")
    print(f"전체 수익률: {(sum_total_value-sum_total_investment)/sum_total_investment*100:.2f}%")


In [215]:
회계(24,"5_10")

-------------Gold------------
총 투자금액: 90.00원
2024-10-20 기준 총 평가 금액: 98.63원
2024-10-20 기준 수익률: 9.59%
-------------KTB------------
총 투자금액: 195.00원
2024-10-20 기준 총 평가 금액: 197.34원
2024-10-20 기준 수익률: 1.20%
-------------200TR------------
총 투자금액: 60.00원
2024-10-20 기준 총 평가 금액: 56.78원
2024-10-20 기준 수익률: -5.37%
-------------USTB------------
총 투자금액: 195.00원
2024-10-20 기준 총 평가 금액: 192.12원
2024-10-20 기준 수익률: -1.48%
-------------Sp500------------
총 투자금액: 60.00원
2024-10-20 기준 총 평가 금액: 63.59원
2024-10-20 기준 수익률: 5.98%
모든 종목의 투자 금액 합계: 600.00원
모든 종목의 최종 평가 금액 합계: 608.46원
전체 수익률: 1.41%


C:\Users\Administrator\AppData\Local\Temp\ipykernel_18272\1412484538.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file_path, index_col='Price', parse_dates=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_18272\1412484538.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file_path, index_col='Price', parse_dates=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_18272\1412484538.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file_path, index_col='Price', parse_dates=True)
C:\Users\Administrator

In [170]:
import pandas as pd

a = "2020-01-01"
a = (pd.to_datetime(a) + pd.DateOffset(days=1)).strftime('%Y-%m-%d')  # 날짜에 하루 추가 후 문자열로 변환
print(a)  # 출력: 2020-01-02
print(type(a))  # <class 'str'>


2020-01-02
<class 'str'>


In [171]:
type("a")

str